In [ ]:
# Install required packages
%pip install chunkr_ai plotly pandas beautifulsoup4 lxml pydantic 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from chunkr_ai import Chunkr
chunkr = Chunkr()
task = await chunkr.upload("https://data.ed.gov/dataset/3683c899-b7b5-4ad4-9b5d-152d197ab389/resource/f5d00bcf-b58e-4914-ba29-a03440e63a9f/download/20action.xlsx")

In [120]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

def parse_table_html(html_content):
    """
    Convert Chunkr HTML table to pandas DataFrame
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    thead = soup.find('thead')
    
    if thead:
        num_header_rows = len(thead.find_all('tr'))
        header_param = list(range(num_header_rows)) if num_header_rows > 1 else 0
    else:
        header_param = 0
    
    tables = pd.read_html(StringIO(html_content), header=header_param)
    return tables[0]

In [121]:
from pydantic import BaseModel, Field, ConfigDict
from chunkr_ai.models import Segment
import uuid

class Table(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    segment: Segment
    df: pd.DataFrame

tables = [
    Table(segment=segment, df=parse_table_html(segment.content))
    for chunk in task.output.chunks
    for segment in chunk.segments
    if segment.segment_type == "Table"
]

In [122]:
class ProgramRow(BaseModel):
    cat_code: str
    appropriation_2019: float
    president_budget_2020: float
    house_2020: float
    senate_mark_2020: float
    appropriation_2020: float
    change_from_2019_amount: float
    change_from_2019_percent: str
    change_from_president_amount: float
    change_from_president_percent: str

    @classmethod
    def from_pd_series(cls, series: pd.Series) -> 'ProgramRow':
        """Convert pandas Series to ProgramRow, handling data cleaning"""
        
        def get_val(key, as_float=False):
            """Get value from series with optional float conversion"""
            try:
                val = series.get(key, 0.0 if as_float else '')
                if isinstance(val, pd.Series):
                    val = val.iloc[0] if not val.empty else (0.0 if as_float else '')
                if as_float:
                    if pd.isna(val): return 0.0
                    return float(str(val).replace(',', '').replace('(', '-').replace(')', '').strip() or '0')
                return '' if pd.isna(val) else str(val)
            except: 
                return 0.0 if as_float else ''
        
        return cls(
            cat_code=get_val('Unnamed: 1_level_0'),
            appropriation_2019=get_val('Unnamed: 2_level_0', True),
            president_budget_2020=get_val('Unnamed: 3_level_0', True),
            house_2020=get_val('Unnamed: 4_level_0', True),
            senate_mark_2020=get_val('Unnamed: 5_level_0', True),
            appropriation_2020=get_val('Unnamed: 6_level_0', True),
            change_from_2019_amount=get_val('Unnamed: 7_level_0', True),
            change_from_2019_percent=get_val('Unnamed: 8_level_0'),
            change_from_president_amount=get_val('Unnamed: 9_level_0', True),
            change_from_president_percent=get_val('Unnamed: 10_level_0')
        )

class ProgramTotal(BaseModel):
    program: str
    row: ProgramRow
 
program_data: list[ProgramTotal] = []

for table in tables:
    df = table.df
    for index, row in df.iterrows():
        if row[0] == "Total" or row[0] == "Total, Appropriation":
            program_name = row.index[0][0]
            program_row = ProgramRow.from_pd_series(row)
            program_data.append(ProgramTotal(program=program_name, row=program_row))

In [123]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def create_clean_budget_dashboard(program_data):
    """Create a clean, focused budget analysis dashboard"""
    
    # Prepare data
    programs = [p.program for p in program_data]
    appropriation_2019 = [p.row.appropriation_2019 for p in program_data]
    appropriation_2020 = [p.row.appropriation_2020 for p in program_data]
    president_budget = [p.row.president_budget_2020 for p in program_data]
    change_from_2019 = [p.row.change_from_2019_amount for p in program_data]
    change_from_president = [p.row.change_from_president_amount for p in program_data]
    change_from_president_pct = [float(str(p.row.change_from_president_percent).replace('%', '').replace('(', '-').replace(')', '').replace('---', '0').strip() or '0') for p in program_data]
    
    # Create clean 2x2 layout
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "📊 Biggest Budget Changes (2019 → 2020)",
            "🎯 Presidential Influence: Top Deviations", 
            "💰 Largest Programs: Before & After",
            "🏛️ Presidential Success Rate"
        ],
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "bar"}]],
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )
    
    # 1. Top Budget Changes - Clean and focused
    impact_data = list(zip(programs, change_from_2019))
    impact_data.sort(key=lambda x: abs(x[1]), reverse=True)
    top_5_changes = impact_data[:5]
    
    # Clean program names
    clean_names = []
    for prog, _ in top_5_changes:
        if "Federal Direct Student Loans" in prog:
            clean_names.append("Federal Student Loans")
        elif "DISCRETIONARY" in prog:
            clean_names.append("Total Discretionary")
        elif "Student Financial Assistance" in prog:
            clean_names.append("Student Financial Aid")
        elif "Education for the Disadvantaged" in prog:
            clean_names.append("Disadvantaged Education")
        else:
            clean_names.append(prog[:25])
    
    colors_1 = ['#2E8B57' if x[1] > 0 else '#DC143C' for x in top_5_changes]
    
    fig.add_trace(
        go.Bar(
            x=clean_names,
            y=[x[1]/1000000 for x in top_5_changes],
            marker_color=colors_1,
            text=[f"${abs(x[1]/1000000):.1f}B" for x in top_5_changes],
            textposition='outside',
            textfont=dict(size=12, color='white'),
            hovertemplate='<b>%{x}</b><br>Change: $%{y:.1f}B<extra></extra>',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Presidential Influence - Where Congress deviated most
    pres_impact = list(zip(programs, change_from_president, change_from_president_pct))
    pres_impact.sort(key=lambda x: abs(x[1]), reverse=True)
    top_5_deviations = pres_impact[:5]
    
    clean_names_2 = []
    for prog, _, _ in top_5_deviations:
        if "Federal Direct Student Loans" in prog:
            clean_names_2.append("Federal Student Loans")
        elif "School Improvement" in prog:
            clean_names_2.append("School Improvement")
        elif "Higher Education" in prog:
            clean_names_2.append("Higher Education")
        elif "Student Financial Assistance" in prog:
            clean_names_2.append("Student Financial Aid")
        else:
            clean_names_2.append(prog[:20])
    
    colors_2 = ['#1E90FF' if x[1] > 0 else '#FF6347' for x in top_5_deviations]
    
    fig.add_trace(
        go.Bar(
            x=clean_names_2,
            y=[x[1]/1000000 for x in top_5_deviations],
            marker_color=colors_2,
            text=[f"{x[2]:.0f}%" for x in top_5_deviations],
            textposition='outside',
            textfont=dict(size=12, color='white'),
            hovertemplate='<b>%{x}</b><br>Deviation: $%{y:.1f}B<br>Percent: %{text}<extra></extra>',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Largest Programs Comparison - Clean side by side
    largest_programs = list(zip(programs, appropriation_2019, appropriation_2020))
    largest_programs.sort(key=lambda x: x[2], reverse=True)
    top_6_programs = largest_programs[:6]
    
    clean_names_3 = []
    for prog, _, _ in top_6_programs:
        if "Federal Direct Student Loans" in prog:
            clean_names_3.append("Student Loans")
        elif "DISCRETIONARY" in prog:
            clean_names_3.append("Total Discretionary")
        elif "Student Financial Assistance" in prog:
            clean_names_3.append("Student Aid")
        elif "Education for the Disadvantaged" in prog:
            clean_names_3.append("Disadvantaged Ed")
        elif "Special Education" in prog:
            clean_names_3.append("Special Education")
        else:
            clean_names_3.append(prog[:15])
    
    fig.add_trace(
        go.Bar(
            x=clean_names_3,
            y=[x[1]/1000000 for x in top_6_programs],
            name='2019',
            marker_color='#B0C4DE',
            offsetgroup=1,
            hovertemplate='<b>%{x}</b><br>2019: $%{y:.1f}B<extra></extra>'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=clean_names_3,
            y=[x[2]/1000000 for x in top_6_programs],
            name='2020',
            marker_color='#4682B4',
            offsetgroup=2,
            hovertemplate='<b>%{x}</b><br>2020: $%{y:.1f}B<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 4. Presidential Success Rate - Simple and clear
    success_high = sum(1 for pct in change_from_president_pct if abs(pct) < 5 and pct != 0)
    success_moderate = sum(1 for pct in change_from_president_pct if 5 <= abs(pct) < 15)
    success_low = sum(1 for pct in change_from_president_pct if abs(pct) >= 15)
    
    fig.add_trace(
        go.Bar(
            x=['Close Match<br>(±5%)', 'Some Deviation<br>(5-15%)', 'Major Deviation<br>(>15%)'],
            y=[success_high, success_moderate, success_low],
            marker_color=['#32CD32', '#FFD700', '#FF4500'],
            text=[success_high, success_moderate, success_low],
            textposition='inside',
            textfont=dict(size=14, color='white', family='Arial Black'),
            hovertemplate='<b>%{x}</b><br>Programs: %{y}<extra></extra>',
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Clean layout
    fig.update_layout(
        height=800,
        title={
            'text': "<b>🎓 Education Budget Analysis 2020</b><br><sup>Presidential Proposals vs Congressional Appropriations</sup>",
            'x': 0.5,
            'font': {'size': 24, 'color': '#2C3E50'}
        },
        font=dict(family="Arial", size=12, color='#2C3E50'),
        plot_bgcolor='white',
        paper_bgcolor='#F8F9FA',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5,
            font=dict(size=12)
        )
    )
    
    # Clean axes - remove clutter
    for i in range(1, 3):
        for j in range(1, 3):
            fig.update_xaxes(
                tickangle=45,
                title_font=dict(size=12),
                tickfont=dict(size=10),
                showgrid=False,
                row=i, col=j
            )
            fig.update_yaxes(
                title_font=dict(size=12),
                tickfont=dict(size=10),
                showgrid=True,
                gridcolor='#E5E5E5',
                gridwidth=1,
                row=i, col=j
            )
    
    # Specific axis labels
    fig.update_yaxes(title_text="Change (Billions $)", row=1, col=1)
    fig.update_yaxes(title_text="Deviation (Billions $)", row=1, col=2)
    fig.update_yaxes(title_text="Appropriation (Billions $)", row=2, col=1)
    fig.update_yaxes(title_text="Number of Programs", row=2, col=2)
    
    return fig

# Create the clean dashboard
clean_dashboard = create_clean_budget_dashboard(program_data)
clean_dashboard.show()
